In [1]:
import pandas as pd
import numpy as np
from haversine import haversine

# Read in the CSV file
data = pd.read_csv('D:\\scriptie\\sensor_analysis\\CNN-training\\csv files\\coordinates_kerkhoflaan.csv')



In [2]:
# Extract the coordinates
coords = data[['lon', 'lat']].values

# Number of sensors
num_sensors = coords.shape[0]

# Create an empty array for the distances
distances = np.zeros((num_sensors, num_sensors))



In [7]:
from tqdm import tqdm

In [9]:
?haversine

In [10]:
# Calculate distances for every pair of points
for i in tqdm(range(num_sensors)):
    for j in tqdm(range(num_sensors)):
        distances[i, j] = haversine(coords[i], coords[j],unit='m')




100%|██████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 8919.83it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 9017.85it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 9017.85it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 9015.70it/s]

100%|███████████████████████████

In [14]:
# Renaming sensors
sensor_names = list(range(1, num_sensors + 1))

# Convert to pandas DataFrame
distances = pd.DataFrame(distances, index=sensor_names, columns=sensor_names)
distances = distances.round(2)

# Mask the lower triangle and diagonal
mask = np.triu(np.ones_like(distances, dtype=bool))
distances = distances.where(mask)

# Show the upper triangular distances matrix
print(distances)

     1      2       3       4       5       6       7       8       9
1  0.0  52.43  106.25  139.98  214.30  307.54  455.28  607.96  809.49
2  NaN   0.00   53.84   88.99  162.04  255.30  402.95  555.59  757.39
3  NaN    NaN    0.00   39.48  108.32  201.56  349.13  501.75  703.74
4  NaN    NaN     NaN    0.00   77.06  169.12  317.00  469.76  670.03
5  NaN    NaN     NaN     NaN    0.00   93.26  241.01  393.74  595.42
6  NaN    NaN     NaN     NaN     NaN    0.00  147.92  300.69  502.21
7  NaN    NaN     NaN     NaN     NaN     NaN    0.00  152.77  355.61
8  NaN    NaN     NaN     NaN     NaN     NaN     NaN    0.00  206.19
9  NaN    NaN     NaN     NaN     NaN     NaN     NaN     NaN    0.00


In [17]:
import geopandas as gpd

In [21]:
from shapely.geometry import Point, LineString
import matplotlib.pyplot as plt


In [30]:
import folium
from geopy import distance


In [42]:
# Calculate center of map
map_center = [data['lat'].mean(), data['lon'].mean()]

# Create the map
m = folium.Map(location=map_center, zoom_start=14)

# Add points for each sensor
for i, row in data.iterrows():
    folium.Marker([row['lat'], row['lon']], popup=row['sensor'], icon=folium.Icon(icon='feed', color='green',prefix='fa')).add_to(m)

# Distances from each sensor to the next
for i in range(len(data) - 1):
    start = (data.loc[i, 'lat'], data.loc[i, 'lon'])
    end = (data.loc[i+1, 'lat'], data.loc[i+1, 'lon'])
    folium.PolyLine([start, end], color="darkblue").add_to(m)
    
    # Compute midpoint and distance
    midpoint = ((start[0] + end[0]) / 2, (start[1] + end[1]) / 2)
    dist = round(distance.distance(start, end).m, 2)  # distance in meters
    folium.Marker(midpoint, popup=f"{dist} m", icon=folium.Icon(icon="arrows-v", prefix='fa', color='gray')).add_to(m)

# Display the map
m.save('map.html')